In [25]:
### resdata - Jupyter Notebook analysis pipeline test
## The testing cells.
#1. Fetch single ring by id and visualise the data
#2. Fetch relevant data from full experiment and visualise as histogram
#3. Fetch relevant data from full experiment and visualise

In [26]:
## Sing in and Authentication
username = 'test_user_main'
password = 'some_password'
email = 'email@test_email.com'
full_name = 'Tomasz Neska'
path = 'http://127.0.0.1:8000/'
file_name = "test.json"
# declaration of variables used

In [27]:
# initialisation cell
import sys
sys.path.insert(0, "/home/splitsky/Desktop/code_repositories/resdata/")
import interface as ui
api = ui.API_interface(path)
api.check_connection()
#purge call - comment out if necessary -> just for development
api.purge_everything()

purged


In [28]:
# make user and populate the database with data
result = api.create_user(username_in=username, password_in=password,email=email,full_name=full_name)
print("User created "  + str(result))

# populate the database
no_of_rings = 5  
no_of_experiments = 2

import testing as t
api.generate_token(username, password)
project_name = "project_test_"
experiment_name = "experiment_test"
ds_size = 2000
for i in range(0,4,1):
    t.generate_optics_project(file_name,[no_of_rings, no_of_experiments], project_name+str(i), experiment_name, author_name=username, size_of_dataset=ds_size)
    project = t.load_file_project(file_name)
    api.insert_project(project)
    
# generate special project for analysis showcase

project_name = "special_project"
experiment_name = "special_experiment_test"
ds_size = 5000

search_value = 15.65 # the value that the analysis is supposed to reveal

t.generate_optics_project_2(file_name, [1,1], project_name, experiment_name, username, ds_size, search_value)
project = t.load_file_project(file_name)
api.insert_project(project)

print("Database populated")
print("")
api.tree_print()
    
    
    

User created True


/home/splitsky/Desktop/code_repositories/resdata/testing.py:169: RuntimeWarning: divide by zero encountered in log
  y = np.log(y)


Database populated

The data tree:
project_test_0
     ->experiment_test 0
         -->ring_no. 0
         -->PL spectrum - ring no.0
         -->TRPL spectrum - ring no.0
         -->Lasing spectrum - ring no.0
         -->ring_no. 1
         -->PL spectrum - ring no.1
         -->TRPL spectrum - ring no.1
         -->Lasing spectrum - ring no.1
         -->ring_no. 2
         -->PL spectrum - ring no.2
         -->TRPL spectrum - ring no.2
         -->Lasing spectrum - ring no.2
         -->ring_no. 3
         -->PL spectrum - ring no.3
         -->TRPL spectrum - ring no.3
         -->Lasing spectrum - ring no.3
         -->ring_no. 4
         -->PL spectrum - ring no.4
         -->TRPL spectrum - ring no.4
         -->Lasing spectrum - ring no.4
     ->experiment_test 1
         -->ring_no. 0
         -->PL spectrum - ring no.0
         -->TRPL spectrum - ring no.0
         -->Lasing spectrum - ring no.0
         -->ring_no. 1
         -->PL spectrum - ring no.1
         -->TRPL sp

In [29]:
# Authenticate user cell
api.generate_token(username, password)

In [ ]:
# Tree print cell
api.tree_print()

In [ ]:
# Pull individual data set by name and plot
dataset = api.return_full_dataset("project_test_3","experiment_test 0", "PL spectrum - ring no.0")
x, y, ey = dataset.data
### plot etc...

import matplotlib.pyplot as plt

plt.plot(x,y,"r+")


In [ ]:
# Pull single ring -> plot dimensions and all spectra
import sys
sys.path.insert(0, "/home/splitsky/Desktop/code_repositories/resdata/")
import interface as ui
import jupyter_driver as dr

api = ui.API_interface(path)
api.check_connection()

api.generate_token(username, password)

api.tree_print()
ring_id = 0
experiment_id = "experiment_test 1"
project_id = "project_test_0"
projects = api.experiment_search_meta(meta_search={"ring_id": ring_id}, experiment_id=experiment_id, project_id=project_id)

for entry in projects:
    print(entry.name)
    
    
for entry in projects:
    dr.plot_from_dataset(entry, entry.data_type, entry.name)
    

In [ ]:
# Pull all rings from experiment and summarise dimensions
import jupyter_driver as dr
# pull full experiment
experiment = api.return_full_experiment(project_name="project_test_0" ,experiment_name="experiment_test 0")
# separate dimensions
datasets = experiment.children
desired_data = []
for ds in datasets:
    if ds.data_type == "dimensions":
        desired_data.append(ds)
        
# summarise the quantities
dr.summarise_dimensions(desired_data)

#TODO: finish this up once matplot lib works

In [ ]:
# Create a group and pull it together to summarise ring dimensions
 
group_name = "very_important_things"
group_permission = "write" # or read is recognised
# project_id, author_name, author_permission, group_name
# the function prints true or false depending whether it succeded on adding the group to the project/experiment/dataset
# _rec versions of the function add to groups recursively

print(api.add_group_to_project_rec("project_test_0", username, group_permission, group_name))
print(api.add_group_to_experiment_rec("project_test_1", "experiment_test 0", username, group_permission, group_name))
print(api.add_group_to_dataset_rec(group_permission, username, group_name, "project_test_2", "experiment_test 1", "TRPL spectrum - ring no.0"))

In [ ]:
# Pull group together to summarise TRPL spectra from the datasets

# use function which can be used with the group name or author name
structure = api.author_query(group_name)

api.tree_print_group(group_name)


In [39]:
import server.datastructure as d

api.tree_print()
author = d.Author(name=username, permission="write")
arr = api.convert_img_to_array(filename="test_cat.jpg")
dataset_in = d.Dataset(name="image", data=arr, meta={"cat_name" : "cat"}, data_type="image", author=[author.dict()],data_headings=[])
api.insert_dataset("project_test_0", "experiment_test 0",dataset_in)
dataset = api.return_full_dataset("project_test_0", "experiment_test 0", "image")

api.convert_array_to_img(dataset.data, "test_cat2.jpg")


name='image' data=[array([[14,  7,  0],
       [15,  8,  0],
       [17, 10,  2],
       ...,
       [42, 13,  0],
       [42, 14,  0],
       [43, 15,  1]], dtype=uint8), array([[20, 13,  5],
       [21, 14,  6],
       [21, 14,  6],
       ...,
       [41, 12,  0],
       [42, 14,  0],
       [43, 15,  1]], dtype=uint8), array([[19, 10,  3],
       [20, 11,  4],
       [20, 11,  4],
       ...,
       [41, 13,  0],
       [42, 14,  0],
       [43, 15,  1]], dtype=uint8), array([[17,  8,  1],
       [18,  9,  2],
       [19, 10,  3],
       ...,
       [41, 13,  0],
       [43, 15,  1],
       [44, 16,  2]], dtype=uint8), array([[23, 12,  6],
       [24, 13,  7],
       [24, 13,  7],
       ...,
       [42, 14,  2],
       [43, 15,  3],
       [44, 16,  4]], dtype=uint8), array([[21, 10,  4],
       [22, 11,  5],
       [24, 13,  7],
       ...,
       [42, 14,  2],
       [42, 14,  2],
       [41, 13,  1]], dtype=uint8), array([[21,  8,  0],
       [22,  9,  1],
       [24, 11,  3],


TypeError: Object of type ndarray is not JSON serializable